In [ ]:
# @title
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import time, warnings
import math
from collections import Counter
from google.colab import drive
#Importing Excel Data File
from google.colab import files
#print('Select Input Data File')
#uploaded = files.upload()
from os import sched_getparam

In [ ]:
drive.mount('/content/drive')
# @title
env = gp.Env(empty=True)
env.setParam('WLSACCESSID','4d7cd1b7-3dfe-4e87-80c7-2ada56221071')
env.setParam('WLSSECRET', 'b307457c-470d-4e18-b9c6-30f10d32ed35')
env.setParam('LICENSEID', 2378987)
env.start()

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Scheduling_Project/new_input_2024_adj.xlsx", sheet_name = None)

In [ ]:
# @title
resident_lv = df["Resident Levels"]
rotation_and_lv = df["Rotations and levels"]
lv_and_rotation = df["Levels and rotations"]
vacations = df["Vacations requests"]

In [ ]:
# @title
#Reorganize data

sub_year_list = list(resident_lv["Variable"])
sub_year_list = [str(item) for item in sub_year_list]

num_resident = np.sum(list(resident_lv['Number']))

lv_and_rotation = dict(lv_and_rotation)
for key in lv_and_rotation:
  lv_and_rotation[key] = lv_and_rotation[key].dropna()
  lv_and_rotation[key] = list(lv_and_rotation[key])
lv_and_rotation = {str(key): value for key, value in lv_and_rotation.items()}


#Set of resident year
Y = [key for key in rotation_and_lv if not isinstance(key, str)]

#Set of subyear in for each resident year
Y_s = {key: [item for item in sub_year_list if str(key) in str(item)] for key in Y}

#Set of year in each tier t
Y_t = {key: Y_s[key] + Y_s[key+1] for key in Y if key < max(Y)}

#Set of resident tier
T = [num for num in Y if num != 5] #assume that resident will have only 5 years in total

#Set of residents I
I = list(range(1,num_resident+1))

#Set of residents i who are in year y
I_y = {}
start = 1

for key, length in zip(sub_year_list, list(resident_lv['Number'])):
    I_y[key] = list(range(start, start + length))
    start += length

#Set of service index
J = list(range(1, len(list(rotation_and_lv['Unnamed: 0']))+1))

#Set of service index for each year
#Services in Jy must be the services that have "or"
J_y = {key: [item for item in lv_and_rotation[key] if " or " in item]for key in sub_year_list}

#Convert service that have "or" to element
J_y = {key: [item.strip() for value in values for item in value.split(' or ')] if isinstance(values, list) else values for key, values in J_y.items()}

services_dict = {service: key for key, service in enumerate(list(rotation_and_lv['Unnamed: 0']))}

#link service name to number of service
J_y = {key: [services_dict[key] for key in values] if values else [] for key, values in J_y.items()}

#Set of months
M = ['July','August','September','October','November','December','January','February','March', 'April','May','June']
#M = ['January','February','March', 'April', 'May','June','July','August','September','October','November','December']


#Set of quarter
Q = ['q1', 'q2', 'q3', 'q4']

#Set of days
D = list(range(1,32))

#Set of services name
Service_name = list(rotation_and_lv['Unnamed: 0'])


#Set of week k
K = ['Period 1', 'Period 2', 'Period 3']
#Input of number of allowable residents for each j service. Assume worst case = 1
# A = [1]* len(Service_name)
# A = np.array(A)

#Input of number of unique job for each resident year
#U = [len(set(item for item in lv_and_rotation[key] if ' or ' not in item)) for key in lv_and_rotation] #Not totally correct for now
#U = [0]* len(sub_year_list)

#Min number of required resident
R = pd.DataFrame()
for i in range(1,(len(Y)+1)):
  R[f"year_{i}_min"] = rotation_and_lv[i]

R = R.values.reshape((len(J),len(Y)))

#Convert all float to int
for y, (set_year, sub_year) in enumerate(Y_s.items()):
    for j in range(len(J)):
        if isinstance(R[j][y], float) and R[j][y] >= 0:
            R[j][y]= int(R[j][y])

#Min number of required resident in tier
C = pd.DataFrame()
for i in range(1,(len(T)+1)):
  C[f"tier_{i}_min"] = rotation_and_lv[f'{i} or {i+1}']

C = C.values.reshape((len(J),len(T)))

for t,(tier, y_tier) in enumerate(Y_t.items()):
    for j in range(len(J)):
        if isinstance(C[j][t], float) and C[j][t] >= 0:
            C[j][t]= int(C[j][t])

required_rotation = {key: [item for item in lv_and_rotation[key] if '*' not in item] for key in lv_and_rotation}
required_rotation = {key: [item for item in required_rotation[key] if ' or ' not in item] for key in required_rotation}
required_rotation = {key: [item for item in Counter(required_rotation[key]).items()] for key in required_rotation}

N_min = [[np.nan for _ in range(len(sub_year_list))] for _ in range(len(J))]
N_max = [[np.nan for _ in range(len(sub_year_list))] for _ in range(len(J))]

for y, key in enumerate(sub_year_list):
    for j, service in enumerate(Service_name):
        for element in required_rotation[key]:
            if element[0] == service:
                N_min[j][y] = element[1]
                N_max[j][y] = element[1]
N_min = np.full((len(J),len(sub_year_list)), N_min)
N_max = np.full((len(J),len(sub_year_list)), N_max)

#for year 1p condition
N_min[:, 1] = 0 #Services for 1p must be at least 0 month
N_max[:, 1] = np.where(~np.isnan(N_max[:,1]), 2, N_max[:, 1]) #Services for 1p must be  max 2 months
N_max[16,1] = 1 #service Elective must be max 1
N_max[:, 1] = np.where(np.isnan(N_max[:,1]), 0, N_max[:, 1]) # Services that are not in list must be max 0
N_min[11:13, 1] = 0 #Service NFA and NFB should not have constraints on number of months required
N_max[11:13, 1] = np.nan #Service NFA and NFB should not have constraints on number of months required

n_months_min = 1
n_months_max = 1

G_min = [n_months_min if J_y[key] else np.nan for key in J_y]
G_max = [n_months_max if J_y[key] else np.nan for key in J_y]




In [ ]:
# @title
# Initialize arrays
Vacation = dict(vacations)

V = np.zeros((len(I), len(D), len(M), len(K)), dtype=int)
H = np.zeros((len(I), len(D), len(M), len(K)), dtype=int)

for i in range(len(I)):
    for num in range(1, 4):
        for vac_type in ["A", "B"]:
            start_key = f'start{num}{vac_type}'
            stop_key = f'stop{num}{vac_type}'
            start_date = Vacation[start_key][i]
            stop_date = Vacation[stop_key][i]
            #print(start_date)
            date_range = pd.date_range(start=start_date, end=stop_date)
            for date in date_range:
                m = date.month + 6 - 1 #Consider Jul as the first month of the year regardless of the year
                if m > 11:
                  m = m - 12 #To convert Jul to the first month, Aug is the second month, etc.
                d = date.day - 1
                k = num -1
                if vac_type == "A":
                    V[i, d, m, k] = 1
                else:
                    H[i, d, m, k] = 1

#V_T = V.transpose((1, 0, 2, 3))
# H_T = H.transpose((1, 0, 2))

In [ ]:
# @title
# Create model
model = gp.Model(env=env)

# Decision variable

#One if resident i work on service j on month m
x = model.addMVar(shape = (len(I),len(J),len(M)), vtype=gp.GRB.BINARY, name="x_ijm")

z = model.addMVar(shape = (len(I),len(J),len(Q)), vtype=gp.GRB.BINARY, name="z_ijq")

#One if resident i had worked on service j on month m
w = model.addMVar(shape = (len(I),len(J)), vtype=gp.GRB.BINARY, name="w_ij")

#One if resident i choose choice A for week k
e = model.addMVar(shape = (len(I),len(K)), vtype=gp.GRB.BINARY, name="e_ik")

#One if resident i actually takes vacation on month m and day d
v = model.addMVar(shape = (len(I),len(D),len(M)), vtype=gp.GRB.BINARY, name="v_idm")

#Num res on vacation on month m day d at service j
o = model.addMVar(shape = (len(J),len(D), len(M)), name="o_jdm")

o_max = model.addVar(name="o_max")

s = model.addMVar(shape = (len(J), len(M)), name="s_jm")

r = model.addMVar(shape = (len(J), len(M)), name="r_jm")

#Set Objective
sum_e = 1*gp.quicksum(gp.quicksum((1-e[i,k] for i in range(len(I)))) for k in range(len(K)))

sum_s = 1*gp.quicksum(gp.quicksum((s[j,m] for j in range(len(J)))) for m in range(len(M)))

sum_r = 1*gp.quicksum(gp.quicksum((r[j,m] for j in range(len(J)))) for m in range(len(M)))


obj =  o_max + sum_e + sum_s + sum_r

model.setObjective(obj, sense=gp.GRB.MINIMIZE)

In [ ]:
# @title
def add_constraints(num):
    if num == 1:
        # Only one service is assigned to one resident on the same month
        for m in range(len(M)):
                for i in range(len(I)):
                    model.addConstr(gp.quicksum(x[i, j, m] for j in range(len(J))) == 1)
        #Vacation Constraints
        for k in range(len(K)):
            for m in range(len(M)):
                for d in range(len(D)):
                    model.addConstr(v[:,d,m] >= e[:,k]*V[:,d,m,k] + (1- e[:,k])*H[:,d,m,k])

        for m in range(len(M)):
            for j in range(len(J)-1):
                for d in range(len(D)):
                    model.addConstr(v[:,d,m]@ x[:,j,m] == o[j,d,m])

        model.addConstr(o_max >= o)
        model.addConstr(o <= 2)

        for i in range(len(I)):
            model.addConstr(gp.quicksum(e[i][k] for k in range(len(K))) >= 1)

        display (widgets.HTML('<h1>Constraint 1 is added</h1>'))
        #print("Constraint 1 is added")
    elif num == 2:
        #Resident in each year must do certain services for at least certain number of months
        for y, year in enumerate(I_y):
            for i_values in I_y[year]:
                i = i_values-1
                for j in range(len(J)):
                    if not math.isnan(N_min[j][y]):
                        model.addConstr(gp.quicksum(x[i, j, m] for m in range(len(M))) >= N_min[j][y])
        display (widgets.HTML('<h1>Constraint 2 is added</h1>'))
        #print("Constraint 2 is added")
    elif num == 3:
        #Resident in each year must do certain services for at most certain number of months
        for y, year in enumerate(I_y):
            for i_values in I_y[year]:
                i = i_values-1
                for j in range(len(J)):
                    if not math.isnan(N_max[j][y]):
                        model.addConstr(gp.quicksum(x[i, j, m] for m in range(len(M))) <= N_max[j][y])
        display (widgets.HTML('<h1>Constraint 3 is added</h1>'))
        #print("Constraint 3 is added")
    elif num == 4:
        #Resident in each year must do one of services from each year's set of services for certain number of months
        for y, (year, j_values) in enumerate(J_y.items()):
            j_y = [j for j in j_values]
            #print("j_y", j_y)
            for i_values in I_y[year]:
                i = i_values -1
                if j_y:
                    #print("i = ", i)
                    model.addConstr(gp.quicksum(w[i, j] for j in j_y) == 1)
                    model.addConstr(gp.quicksum(gp.quicksum(x[i, j, m] for m in range(len(M))) for j in j_y) >= G_min[y])
                    model.addConstr(gp.quicksum(gp.quicksum(x[i, j, m] for m in range(len(M))) for j in j_y) <= G_max[y])
                    for j in j_y:
                        #print("j =", j)
                        model.addConstr(gp.quicksum(x[i, j, m] for m in range(len(M))) >= w[i, j])
                        model.addConstr(gp.quicksum(x[i, j, m] for m in range(len(M))) <= (w[i, j]*G_max[y]))
                        #model.addConstr((w[i, j] == 0) >> (gp.quicksum(x[i, j, m] for m in range(len(M))) == 0))
                       #model.addConstr((w[i, j] == 1) >> (gp.quicksum(x[i, j, m] for m in range(len(M))) >= 1))
        display (widgets.HTML('<h1>Constraint 4 is added</h1>'))
        #print("Constraint 4 is added")
    elif num == 5:
        #Residents from year 1p who got chosen to be in Elective services should be only from Jul to Oct
        #Elective j = 16
        for i_values in I_y['1p']:
            #print(i)
            i = i_values -1
            model.addConstr(gp.quicksum(x[i, 16, m] for m in range(len(M))) >= 0)
            model.addConstr(gp.quicksum(x[i, 16, m] for m in range(len(M))) <= 1)
            for m, month in enumerate(M):
                if month not in ['July','August','September','October']:
                    #print(month, m)
                    model.addConstr(x[i, 16, m] == 0)
        display (widgets.HTML('<h1>Constraint 5 is added</h1>'))
        #print("Constraint 5 is added")
    elif num == 6:
        #Resident from year 2c must do CRS and EGS for consecutive 2 months and not do Trauma for any consecutive month
        for i_values in I_y['2c']:
            i = i_values -1
            for j in [1,3]: #Index for CRS = 3 and EGS = 1
                model.addConstr(x[i,j,0] <= x[i,j,1]) #condition for first month
                model.addConstr(x[i,j,11] <= x[i,j,10]) #condition for last month
                for m in range(1,len(M)-1): # Condition for all month except Jan (begin) and Dec (last)
                  #print(len(M)-1)
                  #print(m)
                  model.addConstr(x[i,j,m-1] + x[i,j,m+1] <= 2 - x[i,j,m])
                  model.addConstr(x[i,j,m-1] + x[i,j,m+1] >= x[i,j,m])

        for i_values in I_y['2c']:
            i = i_values -1
            #print(i)
            #print(Service_name[0])
            for m in range(1,len(M)-1): # Condition for all month except Jan (begin) and Dec (last)
                #print(len(M)-1)
                #print(m)
                model.addConstr(x[i,0,m-1] + x[i,0,m+1] <= 2*(1 - x[i,0,m]))
                model.addConstr(x[i,0,m-1] + x[i,0,m+1] >= 0)
            for m in [0,10]: #condition for first month and last month
                model.addConstr(x[i,0,m] + x[i,0,m+1] <= 1)
        display (widgets.HTML('<h1>Constraint 6 is added</h1>'))
        #print("Constraint 6 is added")
    elif num == 7:
        #Resident from year 3c must do Holy, Peds and MIS for consecutive 2 months and not do Trauma for any consecutive month
        #year 3c for Holy, Peds and MIS
        for i_values in I_y['3c']:
            i = i_values -1
            #print(i)
            for j in [4,7,10]: #Index for Holy=10, Peds=7 and MIS=4
                #print(Service_name[j])
                model.addConstr(x[i,j,0] <= x[i,j,1]) #condition for first month
                model.addConstr(x[i,j,11] <= x[i,j,10]) #condition for last month
                for m in range(1,len(M)-1): # Condition for all month except Jan (begin) and Dec (last)
                  #print(len(M)-1)
                  #print(m)
                  model.addConstr(x[i,j,m-1] + x[i,j,m+1] <= 2 - x[i,j,m])
                  model.addConstr(x[i,j,m-1] + x[i,j,m+1] >= x[i,j,m])

        #year 3c for Trauma
        for i_values in I_y['3c']:
            i = i_values -1
            #print(Service_name[0])
            for m in range(1,len(M)-1): # Condition for all month except Jan (begin) and Dec (last)
                #print(len(M)-1)
                #print(m)
                model.addConstr(x[i,0,m-1] + x[i,0,m+1] <= 2*(1 - x[i,0,m]))
                model.addConstr(x[i,0,m-1] + x[i,0,m+1] >= 0)
            for m in [0,10]: #condition for first month and last month
                model.addConstr(x[i,0,m] + x[i,0,m+1] <= 1)
        display (widgets.HTML('<h1>Constraint 7 is added</h1>'))
        #print("Constraint 7 is added")
    elif num == 8:
        #Resident from year 4 must do Lancaster, Peds, SO, EGS for consecutive 2 months and not do NFS for any consecutive month
        #year 4 for Lancaster, Peds, SO, EGS
        for i_values in I_y['4']:
            i = i_values -1
            #print(i)
            for j in [1,5,7,15]: #Index for Lancaster=15, Peds=7, SO=5, and EGS = 1
                #print(Service_name[j])
                model.addConstr(x[i,j,0] <= x[i,j,1]) #condition for first month
                model.addConstr(x[i,j,11] <= x[i,j,10]) #condition for last month
                for m in range(1,len(M)-1): # Condition for all month except Jan (begin) and Dec (last)
                  #print(len(M)-1)
                  #print(m)
                  model.addConstr(x[i,j,m-1] + x[i,j,m+1] <= 2 - x[i,j,m])
                  model.addConstr(x[i,j,m-1] + x[i,j,m+1] >= x[i,j,m])

        #year 4 for NFS
        for i_values in I_y['4']:
            i = i_values -1
            #print(Service_name[14])
            for m in range(1,len(M)-1): # Condition for all month except Jan (begin) and Dec (last)
                #print(len(M)-1)
                #print(m)
                model.addConstr(x[i,14,m-1] + x[i,14,m+1] <= 2*(1 - x[i,14,m]))
                model.addConstr(x[i,14,m-1] + x[i,14,m+1] >= 0)
            for m in [0,10]: #condition for first month and last month
                model.addConstr(x[i,14,m] + x[i,14,m+1] <= 1)
        display (widgets.HTML('<h1>Constraint 8 is added</h1>'))
        #print("Constraint 8 is added")
    elif num == 9:
        #Resident from year 5 must do CRS, EGS, SO, and MIS for consecutive 3 months
        for i_values in I_y['5']:
            i = i_values -1
            #print("i = ", i)
            for j in [1,3,4,5]: #Index for CRS = 3, EGS = 1, SO = 5, MIS = 4
                #print("j =", j)
                model.addConstr(gp.quicksum(z[i, j, q] for q in range(len(Q))) == 1)
                for q in range(len(Q)):
                    #print("q =", q)
                    for m in [3*q, (3*q)+1, (3*q)+2]:
                        #print("m =", m)
                        model.addConstr(x[i,j,m] == z[i,j,q])
                        #model.addConstr(x[i,j,m] >= z[i,j,q])
        display (widgets.HTML('<h1>Constraint 9 is added</h1>'))
        #print("Constraint 9 is added")
    elif num == 10:
        #No. of resident from each year for each service must be at least minimum no. of required resident
        for y, (set_year, sub_year) in enumerate(Y_s.items()):
            i_indices = []
            for key_year,i_values in I_y.items():
                if key_year in sub_year:
                    i_list = [i - 1 for i in i_values] # take value of i from each year in i list
                    i_indices.extend(i_list) # combine those i list that are in the same Y_s
            #print(i_indices)
            for m in range(len(M)):
                for j in range(len(J)):
                    if isinstance(R[j][y], float) and math.isnan(R[j][y]):
                        # print(R[j][y])
                        # print('add == 0')
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) == 0)
                    elif isinstance(R[j][y], int) and R[j][y] >= 0:
                        #print("Service =", Service_name[j], f"R[{j}][{y}] =", R[j][y])
                        #print(f'add >= {R[j][y]}')
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) >= R[j][y])
                    elif isinstance(R[j][y], str) and '!' in R[j][y]:
                        # print(R[j][y])
                        # print(int(R[j][y][0:len(R[j][y])-1])) #substring and convert to number
                        # print(f'add == {int(R[j][y][0:len(R[j][y])-1])}')
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) == int(R[j][y][0:len(R[j][y])-1]))
                    elif isinstance(R[j][y], str) and '(' in R[j][y]:
                        open_bracket_index = R[j][y].index('(')
                        end_bracket_index = R[j][y].index(')')
                        #print(R[j][y])
                        #print(int(R[j][y][0:open_bracket_index])+int(R[j][y][open_bracket_index +1:end_bracket_index]))
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) >= (int(R[j][y][0:open_bracket_index])+int(R[j][y][open_bracket_index +1:end_bracket_index])))
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) + s[j, m] >= int(R[j][y][0:open_bracket_index]))

        display (widgets.HTML('<h1>Constraint 10 is added</h1>'))
        #print("Constraint 10 is added")
    elif num == 11:
        #No. of resident in each tier for each service each month must be at least minimum no. of compulsory resident
        for t,(tier, y_tier) in enumerate(Y_t.items()):
            i_indices = []
            for y, i_values in I_y.items():
                if y in y_tier: #only y that are in y_t that we will loop and add them. Without this condition, code will loop on every key in I_y which will sum every resident in every year
                    i_list = [i - 1 for i in i_values] # take value of i from each year in i list
                    i_indices.extend(i_list) # combine those i list that are in the same tier
            for m in range(len(M)):
                for j in range(len(J)):
                    if isinstance(C[j][t], float) and math.isnan(C[j][t]):
                        # print(C[j][t])
                        # print('add == 0')
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) == 0)
                    elif isinstance(C[j][t], int) and C[j][t] >= 0:
                        # print(C[j][t])
                        # print(f'add >= {C[j][t]}')
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) + r[j, m] >= C[j][t])
                    elif isinstance(C[j][t], str) and '!' in C[j][t]:
                        # print(C[j][t])
                        # print(int(C[j][t][0:len(C[j][t])-1])) #substring and convert to number
                        # print(f'add == {int(C[j][t][0:len(C[j][t])-1])}')
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) == int(C[j][t][0:len(C[j][t])-1]))
                    elif isinstance(C[j][t], str) and '(' in C[j][t]:
                        open_bracket_index = C[j][t].index('(')
                        end_bracket_index = C[j][t].index(')')
                        #print(C[j][t])
                        #print(int(C[j][t][0:open_bracket_index])+int(C[j][t][open_bracket_index +1:end_bracket_index]))
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) >= (int(C[j][t][0:open_bracket_index])+int(C[j][t][open_bracket_index +1:end_bracket_index])))
                        model.addConstr(gp.quicksum(x[i, j, m] for i in i_indices) + r[j, m] >= int(C[j][t][0:open_bracket_index]))


        display (widgets.HTML('<h1>Constraint 11 is added</h1>'))
        #print("Constraint 11 is added")



In [ ]:
# @title
i = 1
while i < 12:
    add_constraints(i)
    i+=1
    #Optimize
model.optimize()

if model.status == GRB.INFEASIBLE:
        display (widgets.HTML('<h1>Model is infeasible</h1>'))
        # #print(f"Model is infeasible, Constraint {i} cause infeasibility")
        # model.computeIIS()
        # print('\nThe following constraints and variables are in the IIS:')

        # for c in model.getConstrs():
        #   if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

        # for v in model.getVars():
        #   if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
        #   if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

if model.status == GRB.OPTIMAL:
    vacations_table = pd.DataFrame()
    #print(f"Optimal solution = {model.objVal}")



# Create  schedule result table
    schedule_result = pd.DataFrame(index=I, columns=M, dtype=str)
    for m in range(len(M)):
        for i in range(len(I)):
            for j in range(len(J)):
                if x[i,j,m].x > 0.5:
                # Assign the service number to the corresponding cell in the table
                    schedule_result.loc[i+1, M[m]] = Service_name[j]

    result_vacation_table = pd.DataFrame(index=I, columns=K)
    for k in range(len(K)):
        for m in range(len(M)):
            for i in range(len(I)):
                for d in range(len(D)):
                    if e[i,k].x > 0.5:
                        if V[i, d, m, k]* v[i, d, m].x > 0.5:
                            result_vacation_table.loc[i+1, K[k]] = f"Plan A, ({Vacation[f'start{k+1}A'][i].date()} to {Vacation[f'stop{k+1}A'][i].date()})"
                            #result_vacation_table.loc[i+1, K[k]] = "Plan A"

                    else:
                        if H[i, d, m, k]* v[i, d, m].x > 0.5:
                            result_vacation_table.loc[i+1, K[k]] = f"Plan B, ({Vacation[f'start{k+1}B'][i].date()} to {Vacation[f'stop{k+1}B'][i].date()})"
                            #result_vacation_table.loc[i+1, K[k]] = "Plan B"



#Create vacation result table
    Service_name_without_dash = [item for item in Service_name if item != "-"]
    for m in range(len(M)):
        vacation_table = pd.DataFrame(index=D, columns=Service_name_without_dash)
        # vacation_table = pd.DataFrame(index=range(1,32), columns=range(1,19), dtype=int)
        vacation_table.fillna("", inplace=True)  # Initialize all cells as empty strings
        for k in range(len(K)):
            for i in range(len(I)):
                for j in range(len(J)-1):
                    for d in range(len(D)):
                        if e[i,k].x > 0.5:
                            if (V[i, d, m, k] * x[i, j, m].x) > 0.5:
                                cell_value = vacation_table.loc[d+1, Service_name_without_dash[j]]
                                if cell_value != "":
                                    cell_value += ", "  # Add a comma separator if the cell already has a value
                                cell_value += f"Res {i+1}"
                                vacation_table.loc[d+1, Service_name_without_dash[j]] = cell_value
                        else:
                            if (H[i, d, m, k] * x[i, j, m].x) > 0.5:
                                cell_value = vacation_table.loc[d+1, Service_name_without_dash[j]]
                                if cell_value != "":
                                    cell_value += ", "  # Add a comma separator if the cell already has a value
                                cell_value += f"Res {i+1}"
                                vacation_table.loc[d+1, Service_name_without_dash[j]] = cell_value

        vacations_table = pd.concat([vacations_table, vacation_table], axis=0)

#Create wording result
    summary = []
    summary.append(f"Maximum number of residents who take vacation on the same service on the same day (o_max) = {o_max.x}")
    for i in range(len(I)):
        for k in range(len(K)):
            if e[i][k].x < 0.5:
              summary.append(f"Resident {I[i]} have plan B vacation on week {k+1}")

    for y in range(len(Y)):
        for j in range(len(J)):
            for m in range(len(M)):
                if isinstance(R[j][y], str) and '(' in R[j][y]:
                    if s[j][m].x > 0:
                        summary.append(f"Require {s[j][m].x} more residents from year {Y[y]} for service {Service_name[j]} in {M[m]}")

    for t in range(len(T)):
        for j in range(len(J)):
            for m in range(len(M)):
                if isinstance(C[j][t], str) and '(' in C[j][t]:
                    if r[j][m].x > 0:
                        summary.append(f"Require {r[j][m].x} more residents from tier {T[t]} for service {Service_name[j]} in {M[m]}")


    summary_table = pd.DataFrame({"Summary":summary})
    #Insert residents' year make it easier to understand
    resident_years = []
    for key, values in I_y.items():
        resident_years.extend([key] * len(values))
    #schedule_result.insert(0,"Resident",I)
    schedule_result.insert(0,"Year",resident_years)

    #result_vacation_table.insert(0,"Resident",I)
    result_vacation_table.insert(0,"Year",resident_years)

    #Insert each month of the year in vacation table
    #vacations_table.insert(0, "Day",[range(1,32)])
    vacations_table.insert(0,"Month",["Jul",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Aug",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Sep",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Oct",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Nov",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Dec",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Jan",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Feb",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Mar",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Apr",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "May",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,
                                      "Jun",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
                                      ])

    # Export the table to Excel
    #excel_file = "schedule_result.xlsx"
    with pd.ExcelWriter("schedule_result.xlsx") as writer:
     # Write the vacations_table DataFrame to Sheet 1
      schedule_result.to_excel(writer, sheet_name='Schedule', index=True)
      vacations_table.to_excel(writer, sheet_name='Vacation_Check', index=True)
      result_vacation_table.to_excel(writer, sheet_name='Vacation_Plan', index=True)
      summary_table.to_excel(writer, sheet_name='Summary', index=True)

    result = files.download('schedule_result.xlsx')

    # vacations_table.to_excel('Vacation_Table.xlsx')
    # vacations_table_file = files.download('Vacation_Table.xlsx')

    # schedule_result.to_excel('Result_Schedule.xlsx')
    # schedule_result_table_file = files.download("Result_Schedule.xlsx")

    # result_vacation_table.to_excel('Result_Vacation_Plan.xlsx')
    # result_vacation_table_file = files.download("Result_Vacation_Plan.xlsx")
